In [29]:
import wave
import sys
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
import librosa.display
import numpy as np
from scipy import stats
from spafe.frequencies.fundamental_frequencies import FundamentalFrequenciesExtractor
from spafe.frequencies.dominant_frequencies import get_dominant_frequencies 
from scipy.fft import fft,fftfreq,fftshift
from spafe.features.spfeats import extract_feats
import librosa
import os
import pandas as pd
from tqdm import tqdm
import re

In [30]:
df = pd.read_csv("ExtractedFeatures.csv")

In [31]:
df.dropna(inplace=True)

In [32]:
X = df.iloc[:,:-1]

In [33]:
y = df.iloc[:,-1]

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [35]:
pipe = make_pipeline(StandardScaler(),RandomForestClassifier())

In [36]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [37]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [38]:
pipe.score(X_test,y_test)

0.7307898979043524

In [39]:
df = pd.read_csv("ExtractedFeatures2.csv")

In [40]:
X = df.iloc[:,:-1]

In [41]:
y = df.iloc[:,-1]

In [42]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [43]:
pipe2 = make_pipeline(StandardScaler(),RandomForestClassifier())

In [44]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [45]:
pipe2.score(X_test,y_test)

0.7762237762237763

In [46]:
pipe3 = make_pipeline(StandardScaler(),XGBClassifier())


In [47]:
all = VotingClassifier(estimators = [('fi',pipe),('se',pipe2),('xgb',pipe3)],voting = 'soft')

In [48]:
all.fit(X_train, y_train)

/home/lazydog/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:10:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('fi',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('randomforestclassifier',
                                               RandomForestClassifier())])),
                             ('se',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('randomforestclassifier',
                                               RandomForestClassifier())])),
                             ('xgb',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('xgbclassifier',
                                               XGBClassifier(base_...
                                                             int

In [50]:
all.score(X_test,y_test)

0.792361484669177